# Attention-based Neural Machine Translation

**Reference:** Luong, Thang, Hieu Pham and Christopher D. Manning. "Effective Approaches to Attention-based Neural Machine Translation." In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing, pp. 1412-1421. 2015.

* https://www.aclweb.org/anthology/D15-1166/ (main paper reference)
* https://arxiv.org/abs/1508.04025 (alternative paper url)
* https://github.com/tensorflow/nmt (main code reference)
* https://www.tensorflow.org/beta/tutorials/text/nmt_with_attention (alternative code reference)
* https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/seq2seq/python/ops/attention_wrapper.py:2449,2103 (attention implementation)

In [1]:
import Pkg;
packages = ["Knet", "Test", "Printf", "LinearAlgebra", "Random", "CuArrays", "IterTools"]
for p in packages; Pkg.add(p); end

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.ju

In [2]:
using Knet, Test, Base.Iterators, Printf, LinearAlgebra, Random, CuArrays, IterTools

## Code and data from previous projects

Please copy or include the following types and related functions from previous projects:
`Vocab`, `TextReader`, `MTData`, `Embed`, `Linear`, `mask!`, `loss`, `int2str`,
`bleu`.

In [3]:
# Your code here
include("hw3_functions.jl");

## S2S: Sequence to sequence model with attention

In this project we will define, train and evaluate a sequence to sequence encoder-decoder
model with attention for Turkish-English machine translation. The model has two extra
fields compared to `S2S_v1`: the `memory` layer computes keys and values from the encoder,
the `attention` layer computes the attention vector for the decoder.

In [4]:
struct Memory; w; end

struct Attention; wquery; wattn; scale; end

struct S2S
    srcembed::Embed       # encinput(B,Tx) -> srcembed(Ex,B,Tx)
    encoder::RNN          # srcembed(Ex,B,Tx) -> enccell(Dx*H,B,Tx)
    memory::Memory        # enccell(Dx*H,B,Tx) -> keys(H,Tx,B), vals(Dx*H,Tx,B)
    tgtembed::Embed       # decinput(B,Ty) -> tgtembed(Ey,B,Ty)
    decoder::RNN          # tgtembed(Ey,B,Ty) . attnvec(H,B,Ty)[t-1] = (Ey+H,B,Ty) -> deccell(H,B,Ty)
    attention::Attention  # deccell(H,B,Ty), keys(H,Tx,B), vals(Dx*H,Tx,B) -> attnvec(H,B,Ty)
    projection::Linear    # attnvec(H,B,Ty) -> proj(Vy,B,Ty)
    dropout::Real         # dropout probability
    srcvocab::Vocab       # source language vocabulary
    tgtvocab::Vocab       # target language vocabulary
end

## Load pretrained model and data

We will load a pretrained model (16.20 bleu) for code testing.  The data should be loaded
with the vocabulary from the pretrained model for word id consistency.

In [5]:
if !isdefined(Main, :pretrained) || pretrained === nothing
    @info "Loading reference model"
    isfile("s2smodel.jld2") || download("http://people.csail.mit.edu/deniz/comp542/s2smodel.jld2","s2smodel.jld2")
    pretrained = Knet.load("s2smodel.jld2","model")
end
datadir = "datasets/tr_to_en"
if !isdir(datadir)
    @info "Downloading data"
    download("http://www.phontron.com/data/qi18naacl-dataset.tar.gz", "qi18naacl-dataset.tar.gz")
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end
if !isdefined(Main, :tr_vocab)
    BATCHSIZE, MAXLENGTH = 64, 50
    @info "Reading data"
    tr_vocab = pretrained.srcvocab # Vocab("$datadir/tr.train", mincount=5)
    en_vocab = pretrained.tgtvocab # Vocab("$datadir/en.train", mincount=5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    dtrn = MTData(tr_train, en_train, batchsize=BATCHSIZE, maxlength=MAXLENGTH)
    ddev = MTData(tr_dev, en_dev, batchsize=BATCHSIZE)
    dtst = MTData(tr_test, en_test, batchsize=BATCHSIZE)
end

┌ Info: Loading reference model
└ @ Main In[5]:2
┌ Info: Reading data
└ @ Main In[5]:14


MTData(TextReader("datasets/tr_to_en/tr.test", Vocab(Dict("dev" => 1277,"komuta" => 13566,"ellisi" => 25239,"adresini" => 22820,"yüzeyi" => 4051,"paris'te" => 9494,"kafamdaki" => 18790,"yüzeyinde" => 5042,"geçerlidir" => 6612,"kökten" => 7774…), ["<s>", "<unk>", ".", ",", "bir", "ve", "bu", "''", "``", "için"  …  "seçmemiz", "destekleyip", "karşılaştırılabilir", "ördeğin", "gününüzü", "bağışçı", "istismara", "yaşça", "tedci", "fakültesi'nde"], 2, 1, split)), TextReader("datasets/tr_to_en/en.test", Vocab(Dict("middle-income" => 13398,"photosynthesis" => 7689,"polarizing" => 17881,"henry" => 4248,"abducted" => 15691,"rises" => 6225,"hampshire" => 13888,"whiz" => 16835,"cost-benefit" => 13137,"progression" => 5549…), ["<s>", "<unk>", ",", ".", "the", "and", "to", "of", "a", "that"  …  "archaea", "handshake", "brit", "wiper", "heroines", "coca", "exceptionally", "gallbladder", "autopsies", "linguistics"], 2, 1, split)), 64, 9223372036854775807, false, 10, Array{Any,1}[[], [], [], [], [], [


## Part 1. Model constructor

The `S2S` constructor takes the following arguments:
* `hidden`: size of the hidden vectors for both the encoder and the decoder
* `srcembsz`, `tgtembsz`: size of the source/target language embedding vectors
* `srcvocab`, `tgtvocab`: the source/target language vocabulary
* `layers=1`: number of layers
* `bidirectional=false`: whether the encoder is bidirectional
* `dropout=0`: dropout probability

Hints:
* You can find the vocabulary size with `length(vocab.i2w)`.
* If the encoder is bidirectional `layers` must be even and the encoder should have `layers÷2` layers.
* The decoder will use "input feeding", i.e. it will concatenate its previous output to its input. Therefore the input size for the decoder should be `tgtembsz+hidden`.
* Only `numLayers`, `dropout`, and `bidirectional` keyword arguments should be used for RNNs, leave everything else default.
* The memory parameter `w` is used to convert encoder states to keys. If the encoder is bidirectional initialize it to a `(hidden,2*hidden)` parameter, otherwise set it to the constant 1.
* The attention parameter `wquery` is used to transform the query, set it to the constant 1 for this project.
* The attention parameter `scale` is used to scale the attention scores before softmax, set it to a parameter of size 1.
* The attention parameter `wattn` is used to transform the concatenation of the decoder output and the context vector to the attention vector. It should be a parameter of size `(hidden,2*hidden)` if unidirectional, `(hidden,3*hidden)` if bidirectional.

In [6]:
function S2S(hidden::Int, srcembsz::Int, tgtembsz::Int, srcvocab::Vocab, tgtvocab::Vocab;
             layers=1, bidirectional=false, dropout=0)
    # Your code here
#     srcembed::Embed       # encinput(B,Tx) -> srcembed(Ex,B,Tx)
#     encoder::RNN          # srcembed(Ex,B,Tx) -> enccell(Dx*H,B,Tx)
#     memory::Memory        # enccell(Dx*H,B,Tx) -> keys(H,Tx,B), vals(Dx*H,Tx,B)
#     tgtembed::Embed       # decinput(B,Ty) -> tgtembed(Ey,B,Ty)
#     decoder::RNN          # tgtembed(Ey,B,Ty) . attnvec(H,B,Ty)[t-1] = (Ey+H,B,Ty) -> deccell(H,B,Ty)
#     attention::Attention  # deccell(H,B,Ty), keys(H,Tx,B), vals(Dx*H,Tx,B) -> attnvec(H,B,Ty)
#     projection::Linear    # attnvec(H,B,Ty) -> proj(Vy,B,Ty)
#     dropout::Real         # dropout probability
#     srcvocab::Vocab       # source language vocabulary
#     tgtvocab::Vocab       # target language vocabulary
    
    # embedding
    vocab_size_source = length(srcvocab.i2w)
    vocab_size_target = length(tgtvocab.i2w)
    srcembed = Embed(vocab_size_source, srcembsz)
    tgtembed = Embed(vocab_size_target, tgtembsz)
    if bidirectional
        encoder=RNN(srcembsz, hidden, numLayers=1/2*layers, dropout=dropout, bidirectional=bidirectional)
        memory=Memory(param(hidden,2*hidden))
        wattn=param(hidden,3*hidden)        
    else
        memory=Memory(1)
        wattn=param(hidden,2*hidden)
    end
    
    wquery=1
    scale=param(1)
    encoder=RNN(srcembsz, hidden, numLayers=1/2*layers, dropout=dropout, bidirectional=bidirectional)
    decoder=RNN(tgtembsz+hidden, hidden, numLayers=layers, dropout=dropout, bidirectional=bidirectional)
   
    attention=Attention(wquery, wattn, scale)
    
    projection=Linear(hidden, vocab_size_target)    
    S2S(srcembed, encoder, memory, tgtembed, decoder, attention, projection, dropout, srcvocab, tgtvocab)
end

S2S

In [7]:
@testset "Testing S2S constructor" begin
    H,Ex,Ey,Vx,Vy,L,Dx,Pdrop = 8,9,10,length(dtrn.src.vocab.i2w),length(dtrn.tgt.vocab.i2w),2,2,0.2
    m = S2S(H,Ex,Ey,dtrn.src.vocab,dtrn.tgt.vocab;layers=L,bidirectional=(Dx==2),dropout=Pdrop)
    @test size(m.srcembed.w) == (Ex,Vx)
    @test size(m.tgtembed.w) == (Ey,Vy)
    @test m.encoder.inputSize == Ex
    @test m.decoder.inputSize == Ey + H
    @test m.encoder.hiddenSize == m.decoder.hiddenSize == H
    @test m.encoder.direction == Dx-1
    @test m.encoder.numLayers == (Dx == 2 ? L÷2 : L)
    @test m.decoder.numLayers == L
    @test m.encoder.dropout == m.decoder.dropout == Pdrop
    @test size(m.projection.w) == (Vy,H)
    @test size(m.memory.w) == (Dx == 2 ? (H,2H) : ())
    @test m.attention.wquery == 1
    @test size(m.attention.wattn) == (Dx == 2 ? (H,3H) : (H,2H))
    @test size(m.attention.scale) == (1,)
    @test m.srcvocab === dtrn.src.vocab
    @test m.tgtvocab === dtrn.tgt.vocab
end

Test Summary:           | Pass  Total
Testing S2S constructor |   16     16


Test.DefaultTestSet("Testing S2S constructor", Any[], 16, false)

## Part 2. Memory

The memory layer turns the output of the encoder to a pair of tensors that will be used as
keys and values for the attention mechanism. Remember that the encoder RNN output has size
`(H*D,B,Tx)` where `H` is the hidden size, `D` is 1 for unidirectional, 2 for
bidirectional, `B` is the batchsize, and `Tx` is the sequence length. It will be
convenient to store these values in batch major form for the attention mechanism, so
*values* in memory will be a permuted copy of the encoder output with size `(H*D,Tx,B)`
(see `@doc permutedims`). The *keys* in the memory need to have the same first dimension
as the *queries* (i.e. the decoder hidden states). So *values* will be transformed into
*keys* of size `(H,B,Tx)` with `keys = m.w * values` where `m::Memory` is the memory
layer. Note that you will have to do some reshaping to 2-D and back to 3-D for matrix
multiplications. Also note that `m.w` may be a scalar such as `1` e.g. when `D=1` and we
want keys and values to be identical.

In [8]:
function (m::Memory)(x)
    # Your code here
    values=permutedims(x, (1,3,2))
#     print(size(values))
#     print(size(m.w))
    keys=mmul(m.w, values)
    return keys, values
end

You can use the following helper function for scaling and linear transformations of 3-D tensors:

In [9]:
mmul(w,x) = (w == 1 ? x : w == 0 ? 0 : reshape(w * reshape(x,size(x,1),:), (:, size(x)[2:end]...)))

mmul (generic function with 1 method)

In [10]:
@testset "Testing memory" begin
    H,D,B,Tx = pretrained.encoder.hiddenSize, pretrained.encoder.direction+1, 4, 5
    x = KnetArray(randn(Float32,H*D,B,Tx))
    k,v = pretrained.memory(x)
    @test v == permutedims(x,(1,3,2))
    @test k == mmul(pretrained.memory.w, v)
end

Test Summary:  | Pass  Total
Testing memory |    2      2


Test.DefaultTestSet("Testing memory", Any[], 2, false)

## Part 3. Encoder

`encode()` takes a model `s` and a source language minibatch `src`. It passes the input
through `s.srcembed` and `s.encoder` layers with the `s.encoder` RNN hidden states
initialized to `0` in the beginning, and copied to the `s.decoder` RNN at the end. The
steps so far are identical to `S2S_v1` but there is an extra step: The encoder output is
passed to the `s.memory` layer which returns a `(keys,values)` pair. `encode()` returns
this pair to be used later by the attention mechanism.

In [11]:
function encode(s::S2S, src)
    # Your code here
    # init encoder
    s.encoder.h = 0
    s.encoder.c = 0

    # ENCODER
    src_embed_out = s.srcembed(src)  
    #println("source embed out: ", summary(src_embed_out), size(src_embed_out))
    encoder_out = s.encoder(src_embed_out)
    #println("encoder_out:" , summary(encoder_out), size(encoder_out))
    # init decoder with encoder's h and c
    s.decoder.h = s.encoder.h
    s.decoder.c = s.encoder.c
#     print(s.decoder.c)
    s.memory(encoder_out)
end

encode (generic function with 1 method)

In [12]:
@testset "Testing encoder" begin
    src1,tgt1 = first(dtrn)
    key1,val1 = encode(pretrained, src1)
    H,D,B,Tx = pretrained.encoder.hiddenSize, pretrained.encoder.direction+1, size(src1,1), size(src1,2)
    @test size(key1) == (H,Tx,B)
    @test size(val1) == (H*D,Tx,B)
    @test (pretrained.decoder.h,pretrained.decoder.c) === (pretrained.encoder.h,pretrained.encoder.c)
    @test norm(key1) ≈ 1214.4755f0
    @test norm(val1) ≈ 191.10411f0
    @test norm(pretrained.decoder.h) ≈ 48.536964f0
    @test norm(pretrained.decoder.c) ≈ 391.69028f0
end

Test Summary:   | Pass  Total
Testing encoder |    7      7


Test.DefaultTestSet("Testing encoder", Any[], 7, false)

## Part 4. Attention

The attention layer takes `cell`: the decoder output, and `mem`: a pair of (keys,vals)
from the encoder, and computes and returns the attention vector. First `a.wquery` is used
to linearly transform the cell to the query tensor. The query tensor is reshaped and/or
permuted as appropriate and multiplied with the keys tensor to compute the attention
scores. Please see `@doc bmm` for the batched matrix multiply operation used for this
step. The attention scores are scaled using `a.scale` and normalized along the time
dimension using `softmax`. After the appropriate reshape and/or permutation, the scores
are multiplied with the `vals` tensor (using `bmm` again) to compute the context
tensor. After the appropriate reshape and/or permutation the context vector is
concatenated with the cell and linearly transformed to the attention vector using
`a.wattn`. Please see the paper and code examples for details.

Note: the paper mentions a final `tanh` transform, however the final version of the
reference code does not use `tanh` and gets better results. Therefore we will skip `tanh`.

In [13]:
function (a::Attention)(cell, mem)
    # Your code here
    query_tensor=a.wquery*cell
    keys, vals=mem
#     print("query tensor:", size(query_tensor))
#     print("keys:", size(keys))
    query_tensor=permutedims(mmul(a.wquery, cell), (3,1,2)) #shape 512,64,5
    attention_scores=bmm(query_tensor, keys) #shape 5,20,64
#     print("attention:", size(attention_scores))
    scaled_attention_scores=attention_scores* a.scale[1]
    normalized_attention_scores=softmax(scaled_attention_scores, dims=2)
    permuted_vals=permutedims(vals, (2,1,3)) #shape 20, 1024, 64
    
    context_tensor=bmm(normalized_attention_scores, permuted_vals)
#     print("vals:", size(vals))
    permuted_context_tensor=permutedims(context_tensor, (2,3,1))
#     concatenate context_tensor with the cell
    concatenated_context_tensor=vcat(cell, permuted_context_tensor)
    reshaped_context_tensor=reshape(concatenated_context_tensor, (size(a.wattn, 2),:))
#     linearly transformed to the attention vector using a.wattn
    transformed_context_tensor=a.wattn*reshaped_context_tensor
    final_reshaped_context_tensor=reshape(transformed_context_tensor, size(cell))
    return final_reshaped_context_tensor
  
end

In [14]:
@testset "Testing attention" begin
    src1,tgt1 = first(dtrn)
    key1,val1 = encode(pretrained, src1)
    H,B = pretrained.encoder.hiddenSize, size(src1,1)
    Knet.seed!(1)
    x = KnetArray(randn(Float32,H,B,5))
    y = pretrained.attention(x, (key1, val1))
    @test size(y) == size(x)
    @test norm(y) ≈ 808.381f0
end

Test Summary:     | Pass  Total
Testing attention |    2      2


Test.DefaultTestSet("Testing attention", Any[], 2, false)

## Part 5. Decoder

`decode()` takes a model `s`, a target language minibatch `tgt`, the memory from the
encoder `mem` and the decoder output from the previous time step `prev`. After the input
is passed through the embedding layer, it is concatenated with `prev` (this is called
input feeding). The resulting tensor is passed through `s.decoder`. Finally the
`s.attention` layer takes the decoder output and the encoder memory to compute the
"attention vector" which is returned by `decode()`.

In [15]:
function decode(s::S2S, tgt, mem, prev)
    # Your code here
    # DECODER
    tgt_embed_out = s.tgtembed(tgt)
    tgt_embed_out = reshape(tgt_embed_out, size(tgt_embed_out)[1], size(tgt_embed_out)[2], 1)
    #println("tgt_embed_out: ", tgt_embed_out)
    input_feeding=vcat(tgt_embed_out, prev)
    #println("input_feeding: ", input_feeding)
    decoder_output=s.decoder(input_feeding)
    #println("decoder output: ", size(decoder_output))
    attention_vector=s.attention(decoder_output, mem)
    #println("attention vector: ", size(attention_vector))
    return attention_vector
end

decode (generic function with 1 method)

In [16]:
@testset "Testing decoder" begin
    src1,tgt1 = first(dtrn)
    key1,val1 = encode(pretrained, src1)
    H,B = pretrained.encoder.hiddenSize, size(src1,1)
    Knet.seed!(1)
    cell = randn!(similar(key1, size(key1,1), size(key1,3), 1))
    cell = decode(pretrained, tgt1[:,1:1], (key1,val1), cell)
    @test size(cell) == (H,B,1)
    @test norm(cell) ≈ 131.21631f0
end

Test Summary:   | Pass  Total
Testing decoder |    2      2


Test.DefaultTestSet("Testing decoder", Any[], 2, false)

## Part 6. Loss

The loss function takes source language minibatch `src`, and a target language minibatch
`tgt` and returns `sumloss/numwords` if `average=true` or `(sumloss,numwords)` if
`average=false` where `sumloss` is the total negative log likelihood loss and `numwords` is
the number of words predicted (including a final eos for each sentence). The source is first
encoded using `encode` yielding a `(keys,vals)` pair (memory). Then the decoder is called to
predict each word of `tgt` given the previous word, `(keys,vals)` pair, and the previous
decoder output. The previous decoder output is initialized with zeros for the first
step. The output of the decoder at each step is passed through the projection layer giving
word scores. Losses can be computed from word scores and masked/shifted `tgt`.

In [17]:
function (s::S2S)(src, tgt; average=true)
    # Your code here
    
    mem=encode(s, src)
    key, val=mem
    hidden=s.decoder.hiddenSize
    batchSize = size(src,1)
    targetLength = size(tgt,2)
    prev = zeros(Float32, size(s.encoder.h, 1), batchSize, 1)
    if (gpu() >= 0)
        prev = KnetArray(prev)
    end
    sumloss=0
    numwords=0
    
    to_be_masked=copy(tgt[:,2:end])
    mask!(to_be_masked, s.tgtvocab.eos)
    masked=to_be_masked
    
    for i in 1:targetLength-1
        decoder_output=decode(s, tgt[:, i], mem, prev)
        #println("decoder_output: ", size(decoder_output))
        prev=decoder_output
        reshaped_decoder_output=reshape(decoder_output, :, batchSize)
        score=s.projection(reshaped_decoder_output)
        delta_loss, delta_numwords=nll(score, masked[:, i], average=false)
        sumloss=sumloss+delta_loss
        numwords=numwords+delta_numwords
    end
    
    if average
        return sumloss/numwords
    else
        return sumloss,numwords
    end
end

In [18]:
@testset "Testing loss" begin
    src1,tgt1 = first(dtrn)
    @test pretrained(src1,tgt1) ≈ 1.4666592f0
    @test pretrained(src1,tgt1,average=false) == (1949.1901f0, 1329)
end

Testing loss: Test Failed at In[18]:4
  Expression: pretrained(src1, tgt1, average=false) == (1949.1901f0, 1329)
   Evaluated: (1949.19f0, 1329) == (1949.1901f0, 1329)
Stacktrace:
 [1] top-level scope at In[18]:4
 [2] top-level scope at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Test/src/Test.jl:1113
 [3] top-level scope at In[18]:2
Test Summary: | Pass  Fail  Total
Testing loss  |    1     1      2


TestSetException: Some tests did not pass: 1 passed, 1 failed, 0 errored, 0 broken.

## Part 7. Greedy translator

An `S2S` object can be called with a single argument (source language minibatch `src`, with
size `B,Tx`) to generate translations (target language minibatch with size `B,Ty`). The
keyword argument `stopfactor` determines how much longer the output can be compared to the
input. Similar to the loss function, the source minibatch is encoded yield a `(keys,vals)`
pair (memory). We generate the output one time step at a time by calling the decoder with
the last output, the memory, and the last decoder state. The last output is initialized to
an array of `eos` tokens and the last decoder state is initialized to an array of
zeros. After computing the scores for the next word using the projection layer, the highest
scoring words are selected and appended to the output. The generation stops when all outputs
in the batch have generated `eos` or when the length of the output is `stopfactor` times the
input.

In [19]:
function (s::S2S)(src; stopfactor = 3)
    # Your code here
    
    isDone = false
    batch_size = size(src,1)
    input = repeat([s.tgtvocab.eos], batch_size)
    is_all_finished = zeros(batch_size)
    translated_sentences = copy(input)
    max_length_output = 0
    
    mem = encode(s, src)
    
    prev_decoder_output = zeros(Float32, size(s.encoder.h, 1), batch_size, 1)
    if (gpu() >= 0)
        prev_decoder_output = KnetArray(prev_decoder_output)
    end
    input = reshape(input, (length(input), 1))
    
    while (!isDone && max_length_output < stopfactor*size(src,2))        
        
        
        y = decode(s, input, mem, prev_decoder_output)
        prev_decoder_output = y
        
          
        hy, b ,ty = size(y)
        y = reshape(y, (hy, b*ty))
        
        scores = s.projection(y)
        
        output_words = reshape(map(x->x[1], argmax(scores, dims = 1)), batch_size)
        translated_sentences = hcat(translated_sentences, output_words)
       
        max_length_output = size(translated_sentences, 2)
        input = reshape(output_words, (length(output_words), 1))
        
       
        tmp_output_words = copy(output_words)
        tmp_output_words = tmp_output_words .== s.tgtvocab.eos
        is_all_finished += tmp_output_words
        if(sum(is_all_finished.==0)==0)
            isDone = true
        end
    end
    return translated_sentences[:, 2:end]
end   

In [20]:
@testset "Testing translator" begin
    src1,tgt1 = first(dtrn)
    tgt2 = pretrained(src1)
    @test size(tgt2) == (64, 41)
    @test tgt2[1:3,1:3] == [14 25 10647; 37 25 1426; 27 5 349]
end

Test Summary:      | Pass  Total
Testing translator |    2      2


Test.DefaultTestSet("Testing translator", Any[], 2, false)

## Part 8. Training

`trainmodel` creates, trains and returns an `S2S` model. The arguments are described in
comments.

In [21]:
function trainmodel(trn,                  # Training data
                    dev,                  # Validation data, used to determine the best model
                    tst...;               # Zero or more test datasets, their loss will be periodically reported
                    bidirectional = true, # Whether to use a bidirectional encoder
                    layers = 2,           # Number of layers (use `layers÷2` for a bidirectional encoder)
                    hidden = 512,         # Size of the hidden vectors
                    srcembed = 512,       # Size of the source language embedding vectors
                    tgtembed = 512,       # Size of the target language embedding vectors
                    dropout = 0.2,        # Dropout probability
                    epochs = 0,           # Number of epochs (one of epochs or iters should be nonzero for training)
                    iters = 0,            # Number of iterations (one of epochs or iters should be nonzero for training)
                    bleu = false,         # Whether to calculate the BLEU score for the final model
                    save = false,         # Whether to save the final model
                    seconds = 60,         # Frequency of progress reporting
                    )
    @show bidirectional, layers, hidden, srcembed, tgtembed, dropout, epochs, iters, bleu, save; flush(stdout)
    model = S2S(hidden, srcembed, tgtembed, trn.src.vocab, trn.tgt.vocab;
                layers=layers, dropout=dropout, bidirectional=bidirectional)

    epochs == iters == 0 && return model

    (ctrn,cdev,ctst) = collect(trn),collect(dev),collect.(tst)
    traindata = (epochs > 0
                 ? collect(flatten(shuffle!(ctrn) for i in 1:epochs))
                 : shuffle!(collect(take(cycle(ctrn), iters))))

    bestloss, bestmodel = loss(model, cdev), deepcopy(model)
    progress!(adam(model, traindata), seconds=seconds) do y
        devloss = loss(model, cdev)
        tstloss = map(d->loss(model,d), ctst)
        if devloss < bestloss
            bestloss, bestmodel = devloss, deepcopy(model)
        end
        println(stderr)
        (dev=devloss, tst=tstloss, mem=Float32(CuArrays.usage[]))
    end
    save && Knet.save("bestmodel.jld2", "bestmodel", bestmodel)
    bleu && Main.bleu(bestmodel,dev)
    return bestmodel
end

trainmodel (generic function with 1 method)

Train a model: If your implementation is correct, the first epoch should take about 24
minutes on a v100 and bring the loss from 9.83 to under 4.0. 10 epochs would take about 4
hours on a v100. With other GPUs you may have to use a smaller batch size (if memory is
lower) and longer time (if gpu speed is lower).

### Load bestmodel

In [22]:
#Knet.save("bestmodel.jld2", "bestmodel", model)

In [23]:
bestmodel = Knet.load("bestmodel.jld2")["bestmodel"]

S2S(Embed(P(KnetArray{Float32,2}(512,38126))), LSTM(input=512,hidden=512,bidirectional,dropout=0.2), Memory(P(KnetArray{Float32,2}(512,1024))), Embed(P(KnetArray{Float32,2}(512,18857))), LSTM(input=1024,hidden=512,layers=2,dropout=0.2), Attention(1, P(KnetArray{Float32,2}(512,1536)), P(KnetArray{Float32,1}(1))), Linear(P(KnetArray{Float32,2}(18857,512)), P(KnetArray{Float32,1}(18857))), 0.2, Vocab(Dict("ağacından" => 35370,"komuta" => 13566,"ellisi" => 25239,"adresini" => 22820,"yüzeyi" => 4051,"paris'te" => 9494,"kafamdaki" => 18790,"yüzeyinde" => 5042,"geçerlidir" => 6612,"kökten" => 7774…), ["<s>", "<unk>", ".", ",", "bir", "ve", "bu", "''", "``", "için"  …  "seçmemiz", "destekleyip", "karşılaştırılabilir", "ördeğin", "gününüzü", "bağışçı", "istismara", "yaşça", "tedci", "fakültesi'nde"], 2, 1, split), Vocab(Dict("middle-income" => 13398,"photosynthesis" => 7689,"polarizing" => 17881,"henry" => 4248,"abducted" => 15691,"rises" => 6225,"hampshire" => 13888,"whiz" => 16835,"cost-benef

### best bleu score

In [24]:
bleu(bestmodel, ddev)

┣████████████████████┫ [100.00%, 132/132, 00:31/00:31, 4.29i/s]                  ┫ [7.58%, 10/132, 00:02/00:32, 3.79i/s] 


perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LC_CTYPE = "UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").


BLEU = 17.94, 48.9/22.1/12.3/6.1 (BP=1.000, ratio=1.035, hyp_len=85399, ref_len=82502)


It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


"/tmp/jl_yz3Pb7"

Code to sample translations from a dataset

In [25]:
data1 = MTData(tr_dev, en_dev, batchsize=1) |> collect;
function translate_sample(model, data)
    (src,tgt) = rand(data)
    out = bestmodel(src)
    println("SRC: ", int2str(src,model.srcvocab))
    println("REF: ", int2str(tgt,model.tgtvocab))
    println("OUT: ", int2str(out,model.tgtvocab))
end

translate_sample (generic function with 1 method)

Generate translations for random instances from the dev set

In [26]:
translate_sample(bestmodel, data1)

SRC: birinci sırada ne var ?
REF: what 's number one ?
OUT: what is next ?


Code to generate translations from user input

In [27]:
function translate_input(model)
    v = model.srcvocab
    src = [ get(v.w2i, w, v.unk) for w in v.tokenizer(readline()) ]'
    out = model(src)
    println("SRC: ", int2str(src,model.srcvocab))
    println("OUT: ", int2str(out,model.tgtvocab))
end

translate_input (generic function with 1 method)

Generate translations for user input

In [34]:
translate_input(bestmodel)

stdin> derin öğrenme çok zaman alıyor
SRC: derin öğrenme çok zaman alıyor
OUT: deep learning takes a lot of time .


## Competition

The reference model `pretrained` has 16.2 bleu. By playing with the optimization algorithm
and hyperparameters, using per-sentence loss, and (most importantly) splitting the Turkish
words I was able to push the performance to 21.0 bleu. I will give extra credit to groups
that can exceed 21.0 bleu in this dataset.

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*